In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_csv('data/BBAG-long.csv')

In [4]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [5]:
data

,Unnamed: 0,record_id,country,Sex_1F_2M,Education,Barthel,Diabetes_1Y_0N,Hypertension_1Y_0N,Heart_Disease_1Y_0N,Physical_activity_1Y_0N,...,Proficiency,Educ_doct,Educ_upper_sec,Educ_master,Educ_bach,year_bilingues,delta_time,GAP_corrected,GAP_bin,y_pred_corrected
0,0,Share_AT-009869-01,Austria,2,1,10.0,0.0,1.0,0.0,1.0,...,30.6,NaN,73.589996,NaN,NaN,2007,4,-0.957336,0.0,62.042664
1,1,Share_AT-020669-02,Austria,2,1,10.0,0.0,1.0,0.0,1.0,...,30.6,NaN,73.589996,NaN,NaN,2007,4,-1.747516,0.0,60.252484
2,2,Share_AT-027123-02,Austria,2,1,10.0,0.0,1.0,0.0,1.0,...,30.6,NaN,73.589996,NaN,NaN,2007,12,-1.349671,0.0,63.650329
3,3,Share_AT-028320-01,Austria,2,2,10.0,0.0,1.0,0.0,1.0,...,30.6,NaN,73.589996,NaN,NaN,2007,12,-4.838502,0.0,70.161498
4,4,Share_AT-030094-01,Austria,1,2,10.0,0.0,0.0,1.0,1.0,...,30.6,NaN,73.589996,NaN,NaN,2007,4,2.180163,1.0,65.180163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84122,86142,Share_RO-722069-02,Romania,1,2,10.0,0.0,0.0,0.0,1.0,...,21.0,NaN,64.269997,NaN,NaN,2007,12,-4.886865,0.0,60.113135
84123,86143,Share_SE-841359-05,Sweden,1,2,9.0,0.0,0.0,0.0,1.0,...,41.4,NaN,80.400002,NaN,NaN,2007,12,2.974218,1.0,61.974218
84124,86144,Share_SI-234133-03,Slovenia,1,2,10.0,0.0,0.0,0.0,1.0,...,49.2,NaN,73.209999,NaN,NaN,2007,12,-3.610002,0.0,50.389998
84125,86145,Share_SI-352598-02,Slovenia,1,1,10.0,0.0,0.0,1.0,1.0,...,49.2,NaN,73.209999,NaN,NaN,2007,12,4.738801,1.0,63.738801


In [6]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


## RR adding co-vars

In [7]:
covar_list = (
                'gender_equal_l', 'Polution_conc_inv', 'Eq', 'free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est', 'GDP_per_capita',
                'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'
)


In [8]:
results_merge_df_all = data.copy()


In [9]:
def fit_logit_rr(df, features, ycol='GAP_bin'):
    try:
        c_df = df[features + [ycol]].dropna()


        X = c_df[features + ['delta_time']]
        y = c_df[ycol]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        scaler = MinMaxScaler((0.05, 0.95))
        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )

        X_train_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        RR = np.exp(model.params[features[0]])
        return RR

    except ValueError as e:
        msg = str(e)
        if "invalid value detected in weights" in msg or "estimation infeasible" in msg:
            return None
        else:
            raise 


In [10]:
results_all = []

for i in vars_:
    display(i)

    df_base = results_merge_df_all[[i, 'GAP_bin'] + list(covar_list) + ['delta_time']]

    # --- RR base ---
    RR_base = fit_logit_rr(df_base, [i] + ['delta_time'])

    if RR_base is None:
        print(f"[SKIP] {i}: RR base did not converge")
        continue 

    logRR_ref = np.log(RR_base)

    selected = []
    remaining = list(covar_list)
    step = 0

    while len(remaining) > 0:

        deltas = []

        for c in remaining:
            features = [i] + selected + [c]  + ['delta_time']

            RR_new = fit_logit_rr(df_base, features)

            if RR_new is None:
                continue  

            delta = abs(np.log(RR_new) - logRR_ref)

            deltas.append({
                'Feature': i,
                'Step': step + 1,
                'Candidate_covar': c,
                'RR_ref': np.exp(logRR_ref),
                'RR_new': RR_new,
                'Delta_logRR': delta
            })

        if len(deltas) == 0:
            print(f"[STOP] {i}: no covariate could be evaluated at step {step+1}")
            break

        df_deltas = pd.DataFrame(deltas)
        best = df_deltas.sort_values('Delta_logRR').iloc[0]

        selected.append(best['Candidate_covar'])
        remaining.remove(best['Candidate_covar'])
        logRR_ref = np.log(best['RR_new'])
        step += 1

        best['Selected_covar'] = best['Candidate_covar']
        best['Selected_so_far'] = ', '.join(selected)
        results_all.append(best)


'Mono'

'One'

'Two'

'Three'

'Total'

In [11]:
df_stepwise_RR = pd.DataFrame(results_all)
df_stepwise_RR

,Feature,Step,Candidate_covar,RR_ref,RR_new,Delta_logRR,Selected_covar,Selected_so_far
5,Mono,1,cred_elect_est,1.404095,1.400316,0.002695,cred_elect_est,cred_elect_est
5,Mono,2,local_dem_est,1.400316,1.391206,0.006528,local_dem_est,"cred_elect_est, local_dem_est"
6,Mono,3,number_leng_inst,1.391206,1.402069,0.007779,number_leng_inst,"cred_elect_est, local_dem_est, number_leng_inst"
4,Mono,4,inclu_suff_est,1.402069,1.388230,0.009920,inclu_suff_est,"cred_elect_est, local_dem_est, number_leng_ins..."
6,Mono,5,distance,1.388230,1.406928,0.013379,distance,"cred_elect_est, local_dem_est, number_leng_ins..."
3,Mono,6,free_parties_l,1.406928,1.350297,0.041084,free_parties_l,"cred_elect_est, local_dem_est, number_leng_ins..."
0,Mono,7,gender_equal_l,1.350297,1.297275,0.040058,gender_equal_l,"cred_elect_est, local_dem_est, number_leng_ins..."
3,Mono,8,number_stable_inst,1.297275,1.376685,0.059413,number_stable_inst,"cred_elect_est, local_dem_est, number_leng_ins..."
0,Mono,9,Polution_conc_inv,1.376685,1.469711,0.065387,Polution_conc_inv,"cred_elect_est, local_dem_est, number_leng_ins..."
0,Mono,10,Eq,1.469711,1.261723,0.152587,Eq,"cred_elect_est, local_dem_est, number_leng_ins..."
